# Workshop 3 Answer Key

In [ ]:
import numpy as np
import random

def get_data(path):
    f = open(path, 'r')
    
    lines = f.readlines()
    
    training_images = np.zeros((len(lines), 784))
    training_labels = np.zeros((len(lines), 10))
    index = 0
    for line in lines:
        line = line.strip()
        label = int(line[0])
        training_images[index, :] = np.fromstring(line[2:], dtype=int, sep=',')
        training_labels[index, label - 1] = 1.0
        index += 1
        

    f.close()
    
    return training_images / 255, training_labels

training_images, training_labels = get_data("mnist_test.csv")
sample_image, sample_label = training_images[0], training_labels[0]

In [ ]:
# helper method for sigmoid function
def sigmoid(x):
    return np.where(x >= 0, 
                    1 / (1 + np.exp(-x)), 
                    np.exp(x) / (1 + np.exp(x)))




class NeuralNetwork():
    """
    A Fully Connected Neural Network. There are 784 input layer nodes, 12 hidden layer nodes, and 10 output layer
    nodes.
    """
    def __init__(self):
        
        # Arrays to hold node values
        self.N = np.zeros((784, ))
        self.H = np.zeros((12, ))
        self.Z = np.zeros((10, ))
        
        # Arrays to hold weight values (randomly initialized between -1 and 1)
        self.W = 2 * np.random.rand(784, 12) - 1
        self.V = 2 * np.random.rand(12, 10) - 1
        
        # Arrays to hold biases for hidden and output nodes
        self.B = 2 * np.random.rand(12) - 1
        self.C = 2 * np.random.rand(10) - 1
        
        # Arrays to hold the gradients
        self.W_grad = np.zeros((784, 12))
        self.V_grad = np.zeros((12, 10))
        self.B_grad = np.zeros((12, ))
        self.C_grad = np.zeros((10, ))
        

    def fill_input_nodes(self, x):
        """
        Fills input layer with image x.
        
        Parameters:
        x: input vector representing image data, one-dimensional vector
        """

        self.N = x
        
        
    def fill_hidden_nodes(self):
        """
        Fills the hidden layer nodes.
        """
        
        H = np.dot(self.N, self.W) + self.B
        self.H = np.tanh(H)
        
    def fill_output_nodes(self):
        """
        Fills the output layer nodes.
        """

        Z = np.dot(self.H, self.V) + self.C
        self.Z = sigmoid(Z)

    
    def forward(self, x):
        """
        Given an image vector x, fills every node in the network.
        
        Parameters:
        x: input vector representing image data, one-dimensional vector
        """

        self.fill_input_nodes(x)
        self.fill_hidden_nodes()
        self.fill_output_nodes()

    def calculate_loss(self, x, label):
        """
        Given an image vector and its corresponding label vector, calculate the loss.
        
        Parameters:
        x: input vector representing image data, one-dimensional vector
        label: input vector representing the label, a one-dimensional vector. Has a 1 in the position 
               corresponding to the correct answer, and 0s everywhere else.
       
        Returns:
        loss: loss of the network given an image, label pair
        """
        
        out = self.forward(x)
        loss = np.sum((self.Z - label) ** 2)
        return loss
    
    
    #### START WORKING ####
    
    
    def calculate_dL_dVij(self, i, j, label):
        """
        Use the formula you derived to calculate dL/dVi,j.
        Place the result in the appropriate spot in self.V_grad.
        
        Parameters:
        i, j: the indices telling which weight to calculate the partial derivative with respect to
        label: a label vector indicating the "correct" answer for the network
        """
        
        dz = 2 * (self.Z[j] - label[j])
        val = np.dot(self.H, self.V[:, j]) + self.C[j]
        dv = self.H[i] * sigmoid(val) * (1 - sigmoid(val))
        self.V_grad[i, j] = dz * dv
        
    def calculate_V_grad(self, label):
        """
        Use the function you just wrote to fill the entire self.V_grad array.
        """
        
        for i in range(12):
            for j in range(10):
                self.calculate_dL_dVij(i, j, label)
        
    def calculate_dL_dCj(self, j, label):
        """
        Use the formula you derived to calculate dL/dCj.
        Place the result in the appropriate spot in self.C_grad.
        
        Parameters:
        j: the index telling which weight to calculate the partial derivative with respect to
        label: a label vector indicating the "correct" answer for the network
        """
        
        dz = 2 * (self.Z[j] - label[j])
        val = np.dot(self.H, self.V[:, j]) + self.C[j]
        dc = sigmoid(val) * (1 - sigmoid(val))
        self.C_grad[j] = dz * dc
        
    def calculate_C_grad(self, label):
        """
        Use the function you just wrote to fill the entire self.C_grad array.
        """
        
        for j in range(10):
            self.calculate_dL_dCj(j, label)
        
    def calculate_dL_dWij(self, i, j, label):
        """
        Use the formula you derived to calculate dL/dWi,j.
        Place the result in the appropriate spot in self.W_grad.
        
        At this point, it may be helpful to go back into the .calculate_dL_dVij() method and save some values.
        We will need to reuse some values, so it may not make sense to calculate it all over again.
        
        
        Parameters:
        i, j: the indices telling which weight to calculate the partial derivative with respect to
        label: a label vector indicating the "correct" answer for the network
        """
        
        dz = 2 * (self.Z - label)
        val_1 = np.dot(self.H, self.V) + self.C
        dh = self.V[j, :] * sigmoid(val_1) * (1 - sigmoid(val_1))
        dh = np.dot(dz, dh)
        
        val_2 = np.dot(self.N, self.W[:, j]) + self.B[j]
        dw = self.N[i] * (1 / (np.cosh(val_2) ** 2))
        
        self.W_grad[i, j] = dh * dw
        
    def calculate_W_grad(self, label):
        """
        Use the function you just wrote to fill the entire self.W_grad array.
        """
        
        for i in range(784):
            for j in range(12):
                self.calculate_dL_dWij(i, j, label)
        
    def calculate_dL_dBj(self, j, label):
        """
        Use the formula you derived to calculate dL/dBj.
        Place the result in the appropriate spot in self.B_grad.
        
        Parameters:
        j: the index telling which weight to calculate the partial derivative with respect to
        label: a label vector indicating the "correct" answer for the network
        """
        
        dz = 2 * (self.Z - label)
        val_1 = np.dot(self.H, self.V) + self.C
        dh = self.V[j, :] * sigmoid(val_1) * (1 - sigmoid(val_1))
        dh = np.dot(dz, dh)
        
        val_2 = np.dot(self.N, self.W[:, j]) + self.B[j]
        db = (1 / (np.cosh(val_2) ** 2))
        
        self.B_grad[j] = dh * db
        
    def calculate_B_grad(self, label):
        """
        Use the function you just wrote to fill the entire self.B_grad array.
        """
        
        for j in range(12):
            self.calculate_dL_dBj(j, label)
        
    def backpropagate(self, label):
        """
        Perform backprop and fill gradient arrays.
        You should not call this method before calling self.forward()
        
        Parameters:
        label: a label vector indicating the "correct" answer for the network
        """
        
        self.calculate_V_grad(label)
        self.calculate_C_grad(label)
        self.calculate_W_grad(label)
        self.calculate_B_grad(label)
    
    def update(self, lr=0.01):
        """
        Use the full gradient arrays to update the weights and biases.
        
        Parameters:
        lr: the learning rate. I set it to 0.01 by default, but we may later find other values to be better.
        """
        
        self.V -= lr * self.V_grad
        self.C -= lr * self.C_grad
        self.W -= lr * self.W_grad
        self.B -= lr * self.B_grad
        
        
        
        
        
    #### STOP WORKING ####
    
    
    ### NOTE
    # Once we calculate the weight and bias gradients, how can we check that they are correct?
    # Thankfully there is a way to be sure if our code is correct. We take inspiration from the formal 
    # definition of a partial derivative -- the rate of change over an infinitesimally small interval.
    # Thus, we can come up with an estimate of our partial by taking the rate of change over a small interval.
    # Our calculated partial should match.
    
    # Each of these methods will select a random partial from its respective gradient array and compare it
    # with the estimate. It will print both the calculated value and the estimated value.
    # If the two numbers printed are close to the same, then your gradient calculations are correct.
    # If they are very different, your calculations are wrong.
    # Feel free to use these to test your implementations.
    
    
    # Parameters:
    # x: sample image
    # y: sample label

    def check_V_grad(self, x, y, perturb=0.00001):
        randi = (random.randrange(12), random.randrange(10))
        self.forward(x)
        self.calculate_dL_dVij(randi[0], randi[1], y)
        test1 = self.V_grad[randi[0], randi[1]]
        self.clear()
        loss1 = self.calculate_loss(x, y)
        self.clear()
        self.V[randi[0], randi[1]] += perturb
        loss2 = self.calculate_loss(x, y)
        self.clear()
        self.V[randi[0], randi[1]] -= perturb
        
        test2 = (loss2 - loss1) / perturb
        
        print(test1)
        print(test2)
        
    def check_W_grad(self, x, y, perturb=0.00001):
        randi = (random.randrange(784), random.randrange(12))
        self.forward(x)
        self.calculate_dL_dWij(randi[0], randi[1], y)
        test1 = self.W_grad[randi[0], randi[1]]
        self.clear()
        loss1 = self.calculate_loss(x, y)
        self.clear()
        self.W[randi[0], randi[1]] += perturb
        loss2 = self.calculate_loss(x, y)
        self.clear()
        self.W[randi[0], randi[1]] -= perturb
        
        test2 = (loss2 - loss1) / perturb
        
        print(test1)
        print(test2)
        
    
    def check_C_grad(self, x, y, perturb=0.00001):
        randi = random.randrange(10)
        self.forward(x)
        self.calculate_dL_dCj(randi, y)
        test1 = self.C_grad[randi]
        self.clear()
        loss1 = self.calculate_loss(x, y)
        self.clear()
        self.C[randi] += perturb
        loss2 = self.calculate_loss(x, y)
        self.clear()
        self.C[randi] -= perturb
        
        test2 = (loss2 - loss1) / perturb
        
        print(test1)
        print(test2)
        
    def check_B_grad(self, x, y, perturb=0.00001):
        randi = random.randrange(12)
        self.forward(x)
        self.calculate_dL_dBj(randi, y)
        test1 = self.B_grad[randi]
        self.clear()
        loss1 = self.calculate_loss(x, y)
        self.clear()
        self.B[randi] += perturb
        loss2 = self.calculate_loss(x, y)
        self.clear()
        self.B[randi] -= perturb
        
        test2 = (loss2 - loss1) / perturb
        
        print(test1)
        print(test2)
        
        
    def clear(self):
        # Arrays to hold node values
        self.N = np.zeros((784, ))
        self.H = np.zeros((12, ))
        self.Z = np.zeros((10, ))
        
        self.W_grad = np.zeros((784, 12))
        self.V_grad = np.zeros((12, 10))
        self.B_grad = np.zeros((12, ))
        self.C_grad = np.zeros((10, ))
        


Test your code below.

In [ ]:
net = NeuralNetwork()

I put each of the calls for checking different grads in seperate cells. This makes it easy to rerun different grad checks individually. You will want to run each gradient check multiple times to be more confident that all of the partials in the array are calculated correctly.

In [ ]:
print("Test gradient implementation for V:")
net.check_V_grad(sample_image, sample_label)

In [ ]:
print("Test gradient implementation for C:")
net.check_C_grad(sample_image, sample_label)

In [ ]:
print("Test gradient implementation for W:")
net.check_W_grad(sample_image, sample_label)

In [ ]:
print("Test gradient implementation for B:")
net.check_B_grad(sample_image, sample_label)

In [ ]:
net.forward(sample_image)
loss1 = np.sum((net.Z - sample_label) ** 2)
print("The sample loss before backpropagating: {}".format(loss1))

net.backpropagate(sample_label)
net.update(lr=0.1)
net.forward(sample_image)
loss2 = np.sum((net.Z - sample_label) ** 2)
print("The sample loss after backpropagating and updating: {}".format(loss2))

## Important Note:

If you were to try to train your network using the code we just wrote, it would be very slow. This is because the implementation we wrote included many loops and wasted time calculating values that we could have saved during the forward pass of the network. In order to make our code more efficient, we would have to save some more values during forward pass to use for the backprop calculations -- particularly the node values before applying activation function. We would also need to *vectorize* our code -- this involves using matrix operations to encapsulate all of our code, rather than using loops, similar to how we did with the forward pass.

If you feel up to the task, you can read about [vectorization of gradient calculation](https://web.stanford.edu/class/cs224n/readings/gradient-notes.pdf) and give it a shot.